# Pose Error Project
### (Transformer-Encoder)

In [1]:
import logging

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Loading packages ...")
import os
import sys
import time
import pickle
import json

# 3rd party packages

#from tqdm import tqdm
# since we are using it in jupyter notebook
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# Project modules
from options import Options
from running import setup, pipeline_factory, validate, check_progress, NEG_METRICS
from utils import utils
from datasets.data import data_factory, Normalizer
from datasets.datasplit import split_dataset
from models.ts_transformer import model_factory
from models.loss import get_loss_module
from optimizers import get_optimizer

import parser


2023-05-24 10:19:37,719 | INFO : Loading packages ...
2023-05-24 10:19:39,041 | INFO : Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-05-24 10:19:39,042 | INFO : NumExpr defaulting to 8 threads.


# Setup Configurations

In [2]:
# Setting 1 - Single Stage
# Training From Scratch
text = "--output_dir ../experiments/ --comment 'poseErrorPred_from_Scratch_smooth24' \
        --name poseErrorPred_fromScratch_Regression_Selective_3_64 --records_file Regression_records.xls \
        --data_dir ../data/SenseTimeV4_Selective/ --data_class pose \
        --epochs 25 --lr 0.0001 --optimizer RAdam --batch_size 128 \
        --pos_encoding learnable --task regression --print_interval 1\
        --num_layers 3  --num_heads 8 --d_model 64 --dim_feedforward 256 --gpu 2"

In [3]:
# Setting 2 - Two Stages
# Pretrain
'''
text = "--output_dir ../experiments/ --comment 'poseErrorPred_pretrain' \
        --name poseErrorPred_preTrain --records_file Regression_records.xls \
        --data_dir ../data/SenseTimeV4_Selective/ --data_class pose \
        --val_ratio 0.2 --epochs 50 --lr 0.0001 --optimizer RAdam --batch_size 128 \
        --pos_encoding learnable --task regression --print_interval 1"
'''
# Finetune

'''
text = "--output_dir ../experiments --comment 'poseErrorPred_finetune' \
        --name poseErrorPred_finetuned --records_file Regression_records.xls \
        --data_dir ../data/SenseTimeV4_Selective/ --data_class pose \
        --epochs 50 --lr 0.0001 --optimizer RAdam \
        --pos_encoding learnable --d_model 64 \
        --load_model ../experiments/poseErrorPred_preTrain_2023-05-15_15-03-04_vvg/checkpoints/model_best.pth \
        --task regression --change_output --batch_size 128"
'''

'\ntext = "--output_dir ../experiments --comment \'poseErrorPred_finetune\'         --name poseErrorPred_finetuned --records_file Regression_records.xls         --data_dir ../data/SenseTimeV4_Selective/ --data_class pose         --epochs 50 --lr 0.0001 --optimizer RAdam         --pos_encoding learnable --d_model 64         --load_model ../experiments/poseErrorPred_preTrain_2023-05-15_15-03-04_vvg/checkpoints/model_best.pth         --task regression --change_output --batch_size 128"\n'

In [4]:
# Process the setting string
# Generate the config variable
input_text = text.split()
args = Options().parse(input_text)
config = setup(args)

2023-05-24 10:19:39,184 | INFO : Stored configuration file in '../experiments/poseErrorPred_fromScratch_Regression_Selective_3_64_2023-05-24_10-19-39_TGV'


In [5]:
config

{'config_filepath': None,
 'output_dir': '../experiments/poseErrorPred_fromScratch_Regression_Selective_3_64_2023-05-24_10-19-39_TGV',
 'data_dir': '../data/SenseTimeV4_Selective/',
 'load_model': None,
 'resume': False,
 'change_output': False,
 'save_all': False,
 'experiment_name': 'poseErrorPred_fromScratch_Regression_Selective_3_64',
 'comment': "'poseErrorPred_from_Scratch_smooth24'",
 'no_timestamp': False,
 'records_file': 'Regression_records.xls',
 'console': False,
 'print_interval': 1,
 'gpu': '2',
 'n_proc': -1,
 'num_workers': 0,
 'seed': None,
 'limit_size': None,
 'test_only': None,
 'data_class': 'pose',
 'labels': None,
 'test_from': None,
 'test_ratio': 0,
 'val_ratio': 0.2,
 'pattern': None,
 'val_pattern': None,
 'test_pattern': None,
 'normalization': 'standardization',
 'norm_from': None,
 'subsample_factor': None,
 'task': 'regression',
 'masking_ratio': 0.15,
 'mean_mask_length': 3,
 'mask_mode': 'separate',
 'mask_distribution': 'geometric',
 'exclude_feats': N

# Setup Logger

In [6]:
total_epoch_time = 0
total_eval_time = 0

total_start_time = time.time()

In [7]:
# Add file logging besides stdout
file_handler = logging.FileHandler(os.path.join(config['output_dir'], 'output.log'))
logger.addHandler(file_handler)

logger.info('Running:\n{}\n'.format(' '.join(sys.argv)))  # command used to run

2023-05-24 10:19:39,195 | INFO : Running:
/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/tianyi/.local/share/jupyter/runtime/kernel-2e4d2906-4879-4630-ba4d-46f28c4ec180.json



# Setup Training Device

In [8]:
if config['seed'] is not None:
    torch.manual_seed(config['seed'])

device = torch.device('cuda' if (torch.cuda.is_available() and config['gpu'] != '-1') else 'cpu')
logger.info("Using device: {}".format(device))
if device == 'cuda':
    logger.info("Device index: {}".format(torch.cuda.current_device()))

2023-05-24 10:19:39,229 | INFO : Using device: cuda


# Prepare Data

In [9]:
 # Build data
logger.info("Loading and preprocessing data ...")
data_class = data_factory[config['data_class']]
my_data = data_class(config['data_dir'], 
                     pattern=config['pattern'], 
                     n_proc=config['n_proc'], 
                     limit_size=config['limit_size'], 
                     config=config)
feat_dim = my_data.feature_df.shape[1]  # dimensionality of data features
if config['task'] == 'classification':
    validation_method = 'StratifiedShuffleSplit'
    labels = my_data.labels_df.values.flatten()
else:
    validation_method = 'ShuffleSplit'
    labels = None
    
# Modify for the pose error pred
validation_method = 'PoseErrorTimeSplit'
validation_method = 'PoseErrorSequenceSplit'

2023-05-24 10:19:39,234 | INFO : Loading and preprocessing data ...
2023-05-24 10:19:39,236 | INFO : Loading 69 datasets files using 32 parallel processes ...


# Split dataset

In [10]:
# Split dataset
test_data = my_data
test_indices = None  # will be converted to empty list in `split_dataset`, if also test_set_ratio == 0
val_data = my_data
val_indices = []
if config['test_pattern']:  # used if test data come from different files / file patterns
    test_data = data_class(config['data_dir'], pattern=config['test_pattern'], n_proc=-1, config=config)
    test_indices = test_data.all_IDs
if config['test_from']:  # load test IDs directly from file, if available, otherwise use `test_set_ratio`. Can work together with `test_pattern`
    test_indices = list(set([line.rstrip() for line in open(config['test_from']).readlines()]))
    try:
        test_indices = [int(ind) for ind in test_indices]  # integer indices
    except ValueError:
        pass  # in case indices are non-integers
    logger.info("Loaded {} test IDs from file: '{}'".format(len(test_indices), config['test_from']))
if config['val_pattern']:  # used if val data come from different files / file patterns
    val_data = data_class(config['data_dir'], pattern=config['val_pattern'], n_proc=-1, config=config)
    val_indices = val_data.all_IDs

In [11]:
# Note: currently a validation set must exist, either with `val_pattern` or `val_ratio`
# Using a `val_pattern` means that `val_ratio` == 0 and `test_ratio` == 0
if config['val_ratio'] > 0:
    train_indices, val_indices, test_indices = split_dataset(data_indices=my_data.all_IDs,
                                                             validation_method=validation_method,
                                                             n_splits=1,
                                                             validation_ratio=config['val_ratio'],
                                                             test_set_ratio=config['test_ratio'],  # used only if test_indices not explicitly specified
                                                             test_indices=test_indices,
                                                             random_seed=1337,
                                                             labels=labels)
    train_indices = train_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
    val_indices = val_indices[0]  # `split_dataset` returns a list of indices *per fold/split*
else:
    train_indices = my_data.all_IDs
    if test_indices is None:
        test_indices = []

In [12]:
train_cv_indices, val_cv_indices, test_indices = split_dataset(data_indices=my_data.all_IDs,
                                                         validation_method=validation_method,
                                                         n_splits=1,
                                                         validation_ratio=config['val_ratio'],
                                                         test_set_ratio=config['test_ratio'],  # used only if test_indices not explicitly specified
                                                         test_indices=test_indices,
                                                         random_seed=1337,
                                                         labels=labels)
train_cv_indices

[[100100200000,
  100100200001,
  100100200002,
  100100200003,
  100100200004,
  100100200005,
  100100200006,
  100100200007,
  100100200008,
  100100200009,
  100100200010,
  100100200011,
  100100200012,
  100100200013,
  100100200014,
  100100200015,
  100100200016,
  100100200017,
  100100200018,
  100100200019,
  100100200020,
  100100200021,
  100100200022,
  100100200023,
  100100200024,
  100100200025,
  100100200026,
  100100200027,
  100100200028,
  100100200029,
  100100200030,
  100100200031,
  100100200032,
  100100200033,
  100100200034,
  100100200035,
  100100200036,
  100100200037,
  100100200038,
  100100200039,
  100100200040,
  100100200041,
  100100200042,
  100100200043,
  100100200044,
  100100200045,
  100100200046,
  100100200047,
  100100200048,
  100100200049,
  100100200050,
  100100200051,
  100100200052,
  100100200053,
  100100200054,
  100100200055,
  100100200056,
  100100200057,
  100100200058,
  100100200059,
  100100200060,
  100100200061,
  100100

In [13]:
len(train_cv_indices)

8

In [14]:
train_cv_indices[1][-1]

100701001729

In [15]:
len(val_cv_indices[0])

11666

In [16]:
logger.info("{} \t samples may be used for training".format(len(train_cv_indices[0])))
logger.info("{} \t samples will be used for validation".format(len(val_cv_indices[0])))
logger.info("{} \t samples will be used for testing".format(len(test_indices)))

2023-05-24 10:19:54,370 | INFO : 98814 	 samples may be used for training
2023-05-24 10:19:54,371 | INFO : 11666 	 samples will be used for validation
2023-05-24 10:19:54,371 | INFO : 0 	 samples will be used for testing


In [17]:
for cv_idx in range(len(train_cv_indices)):
    print(cv_idx)
    train_indices = train_cv_indices[cv_idx]
    val_indices = val_cv_indices[cv_idx]
    with open(os.path.join(config['output_dir'], 'data_indices.json'), 'w') as f:
        try:
            json.dump({'train_indices': list(map(int, train_indices)),
                       'val_indices': list(map(int, val_indices)),
                       'test_indices': list(map(int, test_indices))}, f, indent=4)
        except ValueError:  # in case indices are non-integers
            json.dump({'train_indices': list(train_indices),
                       'val_indices': list(val_indices),
                       'test_indices': list(test_indices)}, f, indent=4)
    
    ## Pre-process features
    normalizer = None
    if config['norm_from']:
        with open(config['norm_from'], 'rb') as f:
            norm_dict = pickle.load(f)
        normalizer = Normalizer(**norm_dict)
    elif config['normalization'] is not None:
        normalizer = Normalizer(config['normalization'])
        my_data.feature_df.loc[train_indices] = normalizer.normalize(my_data.feature_df.loc[train_indices])
        if not config['normalization'].startswith('per_sample'):
            # get normalizing values from training set and store for future use
            norm_dict = normalizer.__dict__
            with open(os.path.join(config['output_dir'], 'normalization.pickle'), 'wb') as f:
                pickle.dump(norm_dict, f, pickle.HIGHEST_PROTOCOL)
    if normalizer is not None:
        if len(val_indices):
            val_data.feature_df.loc[val_indices] = normalizer.normalize(val_data.feature_df.loc[val_indices])
        if len(test_indices):
            test_data.feature_df.loc[test_indices] = normalizer.normalize(test_data.feature_df.loc[test_indices])
            
    ## Create model
    logger.info("Creating model ...")
    model = model_factory(config, my_data)

    if config['freeze']:
        for name, param in model.named_parameters():
            if name.startswith('output_layer'):
                param.requires_grad = True
            else:
                param.requires_grad = False

    logger.info("Model:\n{}".format(model))
    logger.info("Total number of parameters: {}".format(utils.count_parameters(model)))
    logger.info("Trainable parameters: {}".format(utils.count_parameters(model, trainable=True)))
    
    
    ## Initialize optimizer
    if config['global_reg']:
        weight_decay = config['l2_reg']
        output_reg = None
    else:
        weight_decay = 0
        output_reg = config['l2_reg']

    optim_class = get_optimizer(config['optimizer'])
    optimizer = optim_class(model.parameters(), lr=config['lr'], weight_decay=weight_decay)

    start_epoch = 0
    lr_step = 0  # current step index of `lr_step`
    lr = config['lr']  # current learning step
    # Load model and optimizer state
    if args.load_model:
        model, optimizer, start_epoch = utils.load_model(model, config['load_model'], optimizer, config['resume'],
                                                         config['change_output'],
                                                         config['lr'],
                                                         config['lr_step'],
                                                         config['lr_factor'])
    model.to(device)

    loss_module = get_loss_module(config)
    
    
    if config['test_only'] == 'testset':  # Only evaluate and skip training
        dataset_class, collate_fn, runner_class = pipeline_factory(config)
        test_dataset = dataset_class(test_data, test_indices)

        test_loader = DataLoader(dataset=test_dataset,
                                 batch_size=config['batch_size'],
                                 shuffle=False,
                                 num_workers=config['num_workers'],
                                 pin_memory=True,
                                 collate_fn=lambda x: collate_fn(x, max_len=model.max_len))
        test_evaluator = runner_class(model, test_loader, device, loss_module,
                                            print_interval=config['print_interval'], console=config['console'])
        aggr_metrics_test, per_batch_test = test_evaluator.evaluate(keep_all=True)
        print_str = 'Test Summary: '
        for k, v in aggr_metrics_test.items():
            print_str += '{}: {:8f} | '.format(k, v)
        logger.info(print_str)
        #return
        
    # Initialize data generators
    if config['test_only'] != 'testset':  # Only evaluate and skip training
        dataset_class, collate_fn, runner_class = pipeline_factory(config)
        val_dataset = dataset_class(val_data, val_indices)

        val_loader = DataLoader(dataset=val_dataset,
                                batch_size=config['batch_size'],
                                shuffle=False,
                                num_workers=config['num_workers'],
                                pin_memory=True,
                                collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

        train_dataset = dataset_class(my_data, train_indices)

        train_loader = DataLoader(dataset=train_dataset,
                                  batch_size=config['batch_size'],
                                  shuffle=True,
                                  num_workers=config['num_workers'],
                                  pin_memory=True,
                                  collate_fn=lambda x: collate_fn(x, max_len=model.max_len))

        trainer = runner_class(model, train_loader, device, loss_module, optimizer, l2_reg=output_reg,
                                     print_interval=config['print_interval'], console=config['console'])
        val_evaluator = runner_class(model, val_loader, device, loss_module,
                                           print_interval=config['print_interval'], console=config['console'])

        tensorboard_writer = SummaryWriter(config['tensorboard_dir'])

        best_value = 1e16 if config['key_metric'] in NEG_METRICS else -1e16  # initialize with +inf or -inf depending on key metric
        metrics = []  # (for validation) list of lists: for each epoch, stores metrics like loss, ...
        best_metrics = {}
        
        
    ## Evaluate before training
    aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config, best_metrics,
                                                      best_value, epoch=0)
    metrics_names, metrics_values = zip(*aggr_metrics_val.items())
    metrics.append(list(metrics_values))
    
    
    ## Start training
    logger.info('Starting training...')
    for epoch in tqdm(range(start_epoch + 1, config["epochs"] + 1), desc='Training Epoch', leave=False):
        mark = epoch if config['save_all'] else 'last'
        epoch_start_time = time.time()
        # Training
        aggr_metrics_train = trainer.train_epoch(epoch)  # dictionary of aggregate epoch metrics
        epoch_runtime = time.time() - epoch_start_time
        print()
        print_str = 'Epoch {} Training Summary: '.format(epoch)
        for k, v in aggr_metrics_train.items():
            tensorboard_writer.add_scalar('{}/train'.format(k), v, epoch)
            print_str += '{}: {:8f} | '.format(k, v)
        logger.info(print_str)
        logger.info("Epoch runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(epoch_runtime)))
        total_epoch_time += epoch_runtime
        avg_epoch_time = total_epoch_time / (epoch - start_epoch)
        avg_batch_time = avg_epoch_time / len(train_loader)
        avg_sample_time = avg_epoch_time / len(train_dataset)
        logger.info("Avg epoch train. time: {} hours, {} minutes, {} seconds".format(*utils.readable_time(avg_epoch_time)))
        logger.info("Avg batch train. time: {} seconds".format(avg_batch_time))
        logger.info("Avg sample train. time: {} seconds".format(avg_sample_time))

        # evaluate if first or last epoch or at specified interval
        if (epoch == config["epochs"]) or (epoch == start_epoch + 1) or (epoch % config['val_interval'] == 0):
            aggr_metrics_val, best_metrics, best_value = validate(val_evaluator, tensorboard_writer, config,
                                                                  best_metrics, best_value, epoch)
            metrics_names, metrics_values = zip(*aggr_metrics_val.items())
            metrics.append(list(metrics_values))

        utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(mark)), epoch, model, optimizer)

        # Learning rate scheduling
        if epoch == config['lr_step'][lr_step]:
            utils.save_model(os.path.join(config['save_dir'], 'model_{}.pth'.format(epoch)), epoch, model, optimizer)
            lr = lr * config['lr_factor'][lr_step]
            if lr_step < len(config['lr_step']) - 1:  # so that this index does not get out of bounds
                lr_step += 1
            logger.info('Learning rate updated to: ', lr)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

        # Difficulty scheduling
        if config['harden'] and check_progress(epoch):
            train_loader.dataset.update()
            val_loader.dataset.update()
            
    # Export evolution of metrics over epochs
    header = metrics_names
    metrics_filepath = os.path.join(config["output_dir"], "metrics_" + config["experiment_name"] + ".xls")
    book = utils.export_performance_metrics(metrics_filepath, metrics, header, sheet_name="metrics")

    # Export record metrics to a file accumulating records from all experiments
    utils.register_record(config["records_file"], config["initial_timestamp"], config["experiment_name"],
                          best_metrics, aggr_metrics_val, comment=config['comment'])

    logger.info('Best {} was {}. Other metrics: {}'.format(config['key_metric'], best_value, best_metrics))
    logger.info('All Done!')

    total_runtime = time.time() - total_start_time
    logger.info("Total runtime: {} hours, {} minutes, {} seconds\n".format(*utils.readable_time(total_runtime)))

    #return best_value
    print(best_value)
    
    source_path = config['pred_dir'] + '/best_predictions.npz'
    target_path = config['pred_dir'] + "/best_predictions_{}.npz".format(cv_idx)
    os.rename(source_path, target_path)


0


2023-05-24 10:20:02,139 | INFO : Creating model ...
2023-05-24 10:20:02,165 | INFO : Model:
TSTransformerEncoderClassiregressor(
  (project_inp): Linear(in_features=7, out_features=64, bias=True)
  (pos_enc): LearnablePositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerBatchNormEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (drop

Evaluating Epoch 0   0.0% | batch:         0 of        92	|	loss: 1172.57
Evaluating Epoch 0   1.1% | batch:         1 of        92	|	loss: 749.243
Evaluating Epoch 0   2.2% | batch:         2 of        92	|	loss: 923.352
Evaluating Epoch 0   3.3% | batch:         3 of        92	|	loss: 789.848
Evaluating Epoch 0   4.3% | batch:         4 of        92	|	loss: 223.932
Evaluating Epoch 0   5.4% | batch:         5 of        92	|	loss: 790.005
Evaluating Epoch 0   6.5% | batch:         6 of        92	|	loss: 1412.65
Evaluating Epoch 0   7.6% | batch:         7 of        92	|	loss: 509.785
Evaluating Epoch 0   8.7% | batch:         8 of        92	|	loss: 870.334
Evaluating Epoch 0   9.8% | batch:         9 of        92	|	loss: 340.328
Evaluating Epoch 0  10.9% | batch:        10 of        92	|	loss: 1285.42
Evaluating Epoch 0  12.0% | batch:        11 of        92	|	loss: 363.03
Evaluating Epoch 0  13.0% | batch:        12 of        92	|	loss: 640.968
Evaluating Epoch 0  14.1% | batch:     

2023-05-24 10:20:08,809 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 1.9455015659332275 seconds

2023-05-24 10:20:08,809 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 1.9455015659332275 seconds
2023-05-24 10:20:08,809 | INFO : Avg batch val. time: 0.021146756151448124 seconds
2023-05-24 10:20:08,810 | INFO : Avg sample val. time: 0.00016676680661179732 seconds
2023-05-24 10:20:08,810 | INFO : Epoch 0 Validation Summary: epoch: 0.000000 | loss: 728.205929 | 
/home/tianyi/anaconda3/envs/transformer/lib/python3.8/site-packages/numpy/lib/npyio.py:713: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
2023-05-24 10:20:08,831 | INFO : Starting training...


Evaluating Epoch 0  98.9% | batch:        91 of        92	|	loss: 1172.99



...

2023-05-24 10:23:28,972 | INFO : Validation runtime: 0.0 hours, 0.0 minutes, 1.104506254196167 seconds

2023-05-24 10:23:28,972 | INFO : Avg val. time: 0.0 hours, 0.0 minutes, 1.2869288325309753 seconds
2023-05-24 10:23:28,972 | INFO : Avg batch val. time: 0.013988356875336689 seconds
2023-05-24 10:23:28,973 | INFO : Avg sample val. time: 0.00011031448933061679 seconds
2023-05-24 10:23:28,973 | INFO : Epoch 12 Validation Summary: epoch: 12.000000 | loss: 4.307543 | 


Evaluating Epoch 12  85.9% | batch:        79 of        92	|	loss: 5.29055
Evaluating Epoch 12  87.0% | batch:        80 of        92	|	loss: 5.1877
Evaluating Epoch 12  88.0% | batch:        81 of        92	|	loss: 6.19263
Evaluating Epoch 12  89.1% | batch:        82 of        92	|	loss: 1.97303
Evaluating Epoch 12  90.2% | batch:        83 of        92	|	loss: 2.26339
Evaluating Epoch 12  91.3% | batch:        84 of        92	|	loss: 4.14302
Evaluating Epoch 12  92.4% | batch:        85 of        92	|	loss: 6.19046
Evaluating Epoch 12  93.5% | batch:        86 of        92	|	loss: 3.94745
Evaluating Epoch 12  94.6% | batch:        87 of        92	|	loss: 4.30146
Evaluating Epoch 12  95.7% | batch:        88 of        92	|	loss: 6.09488
Evaluating Epoch 12  96.7% | batch:        89 of        92	|	loss: 6.4531
Evaluating Epoch 12  97.8% | batch:        90 of        92	|	loss: 2.42761
Evaluating Epoch 12  98.9% | batch:        91 of        92	|	loss: 0.278697

Training Epoch 13   0.0% 

Training Epoch 13  14.0% | batch:       108 of       772	|	loss: 0.465403
Training Epoch 13  14.1% | batch:       109 of       772	|	loss: 0.410416
Training Epoch 13  14.2% | batch:       110 of       772	|	loss: 0.445914
Training Epoch 13  14.4% | batch:       111 of       772	|	loss: 0.399168
Training Epoch 13  14.5% | batch:       112 of       772	|	loss: 0.496543
Training Epoch 13  14.6% | batch:       113 of       772	|	loss: 0.445918
Training Epoch 13  14.8% | batch:       114 of       772	|	loss: 0.633289
Training Epoch 13  14.9% | batch:       115 of       772	|	loss: 0.490364
Training Epoch 13  15.0% | batch:       116 of       772	|	loss: 0.403562
Training Epoch 13  15.2% | batch:       117 of       772	|	loss: 0.526389
Training Epoch 13  15.3% | batch:       118 of       772	|	loss: 0.569964
Training Epoch 13  15.4% | batch:       119 of       772	|	loss: 0.520461
Training Epoch 13  15.5% | batch:       120 of       772	|	loss: 0.507012
Training Epoch 13  15.7% | batch:     

Training Epoch 13  29.3% | batch:       226 of       772	|	loss: 0.583471
Training Epoch 13  29.4% | batch:       227 of       772	|	loss: 0.479347
Training Epoch 13  29.5% | batch:       228 of       772	|	loss: 0.541638
Training Epoch 13  29.7% | batch:       229 of       772	|	loss: 0.425461
Training Epoch 13  29.8% | batch:       230 of       772	|	loss: 0.366412
Training Epoch 13  29.9% | batch:       231 of       772	|	loss: 0.662202
Training Epoch 13  30.1% | batch:       232 of       772	|	loss: 0.493608
Training Epoch 13  30.2% | batch:       233 of       772	|	loss: 0.41811
Training Epoch 13  30.3% | batch:       234 of       772	|	loss: 0.383404
Training Epoch 13  30.4% | batch:       235 of       772	|	loss: 0.782639
Training Epoch 13  30.6% | batch:       236 of       772	|	loss: 0.42275
Training Epoch 13  30.7% | batch:       237 of       772	|	loss: 0.488684
Training Epoch 13  30.8% | batch:       238 of       772	|	loss: 0.461924
Training Epoch 13  31.0% | batch:       

Training Epoch 13  44.3% | batch:       342 of       772	|	loss: 0.493555
Training Epoch 13  44.4% | batch:       343 of       772	|	loss: 0.454757
Training Epoch 13  44.6% | batch:       344 of       772	|	loss: 0.469336
Training Epoch 13  44.7% | batch:       345 of       772	|	loss: 0.61576
Training Epoch 13  44.8% | batch:       346 of       772	|	loss: 0.657864
Training Epoch 13  44.9% | batch:       347 of       772	|	loss: 0.475778
Training Epoch 13  45.1% | batch:       348 of       772	|	loss: 0.659907
Training Epoch 13  45.2% | batch:       349 of       772	|	loss: 0.390553
Training Epoch 13  45.3% | batch:       350 of       772	|	loss: 0.578506
Training Epoch 13  45.5% | batch:       351 of       772	|	loss: 0.572704
Training Epoch 13  45.6% | batch:       352 of       772	|	loss: 0.543027
Training Epoch 13  45.7% | batch:       353 of       772	|	loss: 0.638868
Training Epoch 13  45.9% | batch:       354 of       772	|	loss: 0.48437
Training Epoch 13  46.0% | batch:       

In [18]:
os.rename(source_path, target_path)

In [ ]:
source_path = config['pred_dir'] + '/best_predictions.npz'
target_path = config['pred_dir'] + "/best_predictions_{}.npz".format(cv_idx)

# Evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
idx = 0

In [ ]:
total_path = config['pred_dir'] + "/best_predictions_{}.npz".format(idx)
total_path

In [ ]:
config['pred_dir']

In [ ]:
config['output_dir']

In [ ]:
pred = np.load(total_path, allow_pickle=True)
pred.files

In [ ]:
y = np.concatenate(pred["targets"], axis=0)
y_pred = np.concatenate(pred["predictions"], axis=0)
IDs = np.concatenate(pred["IDs"], axis=0)
y_pred.shape

In [ ]:
def get_mape(y, y_pred):
    err = y - y_pred
    return np.mean(np.abs(err)/y)

def get_mse(y, y_pred):
    err = y - y_pred
    return np.mean(np.square(err))

In [ ]:
get_mse(y, y_pred)

In [ ]:
get_mape(y, y_pred)

In [ ]:
y.shape

In [ ]:
len(val_indices)

In [ ]:
def visualize_output(y, y_pred, title=' '):
    fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(14, 6))
    fig.tight_layout(pad=3.0)
    
    ax0.plot(y, color='g', label='truth')
    ax0.set_title(title)
    ax0.set_xlabel('Step')
    ax0.set_ylabel('Error')
    ax0.grid()
    ax0.legend()
    
    ax1.plot(y, color='g', label='truth')
    ax1.plot(y_pred, color='b', alpha=0.7, label='predict')
    mse = get_mse(y, y_pred)
    mape = get_mape(y, y_pred)
    ax1.set_title(title+"- mse: {:.5f} | mape: {:.5f}".format(mse, mape))
    ax1.set_xlabel('Step')
    ax1.set_ylabel('Error')
    ax1.grid()
    ax1.legend()
    plt.show()

In [ ]:
visualize_output(y, y_pred, title='Prediction on SenseTime Dataset')

In [ ]:
idx += 1